In [ ]:
import inspect

import matplotlib.pyplot as plt
import numpy as np

Equation 1 from [Lorenz (1996)](https://www.ecmwf.int/en/elibrary/10829-predictability-problem-partly-solved) is:
\begin{align}
\frac{d}{dt} X_k &= - X_{k-2} X_{k-1} + X_{k-1} X_{k+1} - X_k + F
\end{align}
where $F$ is a constant independent of $k=0,1,2,\ldots,K-1$ (using python indexing starting at k=0). The $k$ index is periodic so that, for example, $k=K$ is referring to $k=0$ and $k=-1$ is referring to $k=K-1$.

The following function returns the rate of change $\frac{d}{dt} X_j$:

In [ ]:
from L96_model import L96_eq1_xdot

print(inspect.getsource(L96_eq1_xdot))

We can use different numerical schemes for time integration (to be investigated). Below is an example using 4th order Runge-Kutta. 

In [ ]:
from L96_model import RK4

print(inspect.getsource(RK4))

To use the model we add a "time loop", iterating over consecutive values of time instances, passing forward the model state from the $n$-th time instance to the $n+1$ time instance. We assume a fixed time step, $\Delta t$ so that $t(n) = n\Delta t$.

In [ ]:
from L96_model import integrate_L96_1t

print(inspect.getsource(integrate_L96_1t))

We need some non-uniform initial conditions. These are often chosen as random numbers but in order to have reproducible results we prescribe a simple periodic function (a sine function would do but a cubic one is more precisely reproducible for the interval [-1,1]).

In [ ]:
K = 10  # Number of degrees of freedom
nt = 250  # Number of time steps
dt = 0.02  # time step
F = 8.0  # Model parameter

# Initial conditions
k = np.arange(K)  # For coordinate in plots
s = 2 * (0.5 + k) / K - 1  # A symmetric coordinate from -1..+1
Xinit = s * (s - 1) * (s + 1)  # Initial conditions

# "Run" model
X, t = integrate_L96_1t(Xinit, F, dt, nt)

plt.figure(figsize=(12, 10))
plt.subplot(221)
# Snapshot of X[k]
plt.plot(k, X[-1])
plt.xlabel("k")
plt.title("$X_k(t=N\Delta t)$")
plt.plot(k, Xinit, "k:")
plt.subplot(222)
# Sample time-series X[0](t)
plt.plot(t, X[:, 0])
plt.xlabel("t")
plt.title("$X_0(t)$")
plt.subplot(223)
# Full model history
plt.contourf(k, t, X)
plt.colorbar(orientation="horizontal")
plt.xlabel("k")
plt.ylabel("t")
plt.title("$X_k(t)$")
plt.subplot(224)
# Metric, total model energy
plt.plot(t, (X**2).sum(axis=-1))
plt.xlabel("t")
plt.title("$\sum_k X_k^2(t)$");

In [ ]:
Xrk4_001, t001 = integrate_L96_1t(Xinit, F, 0.01, 500)
Xrk4_002, t002 = integrate_L96_1t(Xinit, F, 0.02, 250)
Xrk4_005, t005 = integrate_L96_1t(Xinit, F, 0.05, 100)
Xrk4_019, t019 = integrate_L96_1t(Xinit, F, 0.19, 26)

plt.plot(t001, Xrk4_001[:, 0], label="$\Delta t$=0.01")
plt.plot(t002, Xrk4_002[:, 0], label="$\Delta t$=0.02")
plt.plot(t005, Xrk4_005[:, 0], ".-", label="$\Delta t$=0.05")
plt.plot(t019, Xrk4_019[:, 0], ".-", label="$\Delta t$=0.19")
plt.legend()
plt.xlabel("Time, t")
plt.ylabel("$X_0(t)$")
plt.title("Time resolution");

In [ ]:
from L96_model import RK2, EulerFwd

dt, nt = 0.01, 500
Xrk4, t = integrate_L96_1t(Xinit, F, dt, nt, method=RK4)
Xrk2, _ = integrate_L96_1t(Xinit, F, dt, nt, method=RK2)
Xef, _ = integrate_L96_1t(Xinit, F, dt, nt, method=EulerFwd)

plt.plot(t, Xrk4[:, 0], label="RK4")
plt.plot(t, Xrk2[:, 0], label="RK2")
plt.plot(t, Xef[:, 0], label="Euler")
plt.legend()
plt.xlabel("Time, t")
plt.ylabel("$X_0(t)$")
plt.title("Role of time-stepping accuracy");

In [ ]:
def l2nrm(X, Xtrth):
    return np.sqrt(((X[-1, :] - Xtrth[-1, :]) ** 2).mean())


NT, Lt = [16, 20, 50, 100, 200, 500, 2000, 4000], 3
D, E4, E2, E1 = (
    np.zeros(len(NT)),
    np.zeros(len(NT)),
    np.zeros(len(NT)),
    np.zeros(len(NT)) * np.nan,
)
Xtrth, t = integrate_L96_1t(Xinit, F, 0.25 / NT[-1], Lt * 4 * NT[-1], method=RK4)
for i in range(len(NT)):
    dt = 1.0 / NT[i]
    D[i] = dt
    X, t = integrate_L96_1t(Xinit, F, dt, Lt * NT[i], method=RK4)
    E4[i] = l2nrm(X, Xtrth)
    X, t = integrate_L96_1t(Xinit, F, dt, Lt * NT[i], method=RK2)
    E2[i] = l2nrm(X, Xtrth)
    if dt < 0.04:
        X, t = integrate_L96_1t(Xinit, F, dt, Lt * NT[i], method=EulerFwd)
        E1[i] = l2nrm(X, Xtrth)

plt.loglog(D, E4, "x", label="RK4")
plt.loglog(D, E2, "o", label="RK2")
plt.loglog(D, E1, ".", label="Euler")
plt.loglog(D, 8e2 * D, "k:", label="$\Delta t$")
plt.loglog(D, 1e3 * D**2, "k-.", label="$\Delta t^2$")
plt.loglog(D, 1e4 * D**4, "k--", label="$\Delta t^4$")
plt.legend()
plt.grid()
plt.xlabel("$\Delta t$")
plt.ylabel("$l_2$ norm")
plt.title("Convergence rates");